# Máximo de una funcion

In [18]:
from math import sin, pi
from typing import Callable

Creamos una función que nos devuelve el valor de la función en un punto dado.

In [19]:
def f(x: int) -> float:
    """La función a ser maximizada"""
    return x * sin(10*pi*x) + 1

Para trabajar con cadenas de longitud 22 binarias, preferiremos esta vez usar listas en lugar de numpy.
Creamos una funcion que convierte de listas de 22 bits a su representación entera:

In [20]:
def decimal_de(cadena: list) -> float:
    """Da la representación decimal del gen"""
    num = 0
    for i in range(len(cadena) - 1, -1, -1):
        num += cadena[len(cadena) - 1 - i] * 2**i
    num = -1 + (3 / (2**22 - 1)) * num
    return num


In [21]:
# Prueba de funcionamiento:
import random
import timeit
cadena = [0,1,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,1,0,1,0]
#generar 50000 cadenas de 22 bits

# print(timeit.timeit(lambda: f(decimal_de(cadena)), number=10000000))

## Población inicial
Para crear una población escogemos que tenga 50 individuos, y para cada uno de ellos generamos una lista de 22 bits aleatorios.

In [22]:
INDIVIDUOS = 50

In [23]:
def crear_poblacion(tamano: int) -> list:
    """Crea una población de tamaño individuos para el problema de la maximización de la función aptitud."""
    poblacion = []
    for i in range(tamano):
        cadena = [random.randint(0, 1) for _ in range(22)]
        poblacion.append(cadena)
    return poblacion

In [24]:
población = crear_poblacion(INDIVIDUOS)

## Función de evaluación
La aptitud será la función que queremos maximizar:

In [25]:
def aptitud_maximizar(individuo: list) -> float:
    """Aptitud de un individuo"""
    return f(decimal_de(individuo))

## Operadores genéticos

### Selección
Para seleccionar los padres usamos el método de la ruleta con pesos.

In [26]:
def seleccionar_ruleta(poblacion: list, n: int, fun_aptitud: Callable = aptitud_maximizar) -> list:
    """Selecciona n individuos de la población usando el método de la ruleta.
    :param poblacion: matriz de individuos
    :param n: número de individuos a seleccionar
    :param fun_aptitud: función para calcular la aptitud de un individuo
    :return: matriz de individuos seleccionados"""

    aptitudes = [fun_aptitud(individuo) for individuo in poblacion]
    poblacion_seleccionada = []
    aptitudes_acumuladas = [sum(aptitudes[: i + 1]) for i in range(len(aptitudes))]
    aptitud_total = aptitudes_acumuladas[-1]

    # Para cada progenitor a seleccionar
    for _ in range(n):
        valor = random.randint(0, aptitud_total)

        seleccionado = 0

        while not aptitudes_acumuladas[seleccionado] >= valor:
            seleccionado += 1
        poblacion_seleccionada.append(poblacion[seleccionado])
    return poblacion_seleccionada

### Crossover

In [27]:
def crossover(
    progenitores: list,
    probabilidad: float = 1,
) -> list:
    """Produce 2 nuevos individuos a partir de crossover si la probabilidad lo permite. Si no se produce, se retornan los progenitores.
    :param progenitores: matriz de progenitores
    :return: matriz de individuos con los nuevos individuos"""

    if random.uniform(0, 1) <= probabilidad:
        # Me quedo con dos progenitores al azar
        progenitores = random.sample(list(progenitores), 2)
        # Elijo un punto de corte
        punto_corte = random.randint(1, len(progenitores[0]) - 1)
        # Creo a los hijos
        hijo1 = []
        hijo1.append(progenitores[0][:punto_corte])
        hijo1.append(progenitores[1][punto_corte:])
        hijo2 = []
        hijo2.append(progenitores[1][:punto_corte])
        hijo2.append(progenitores[0][punto_corte:])
        # Los devuelvo como ndarray
        return [hijo1, hijo2]
    else:
        return progenitores


### Mutación

In [28]:
def mutacion(individuo: list, probabilidad: float, max_genes_a_mutar: int = 1) -> None:
    """Realiza la mutación de un individuo con una probabilidad dada.
    :param individuo: lista que contiene los genes del individuo
    :param probabilidad: probabilidad de mutación"""
    if random.uniform() <= probabilidad:
        num_genes_mutar = random.randint(0, max_genes_a_mutar)
        indices_a_mutar = random.sample(range(len(individuo)), num_genes_mutar)
        for i in indices_a_mutar:
            individuo[i] = random.choice([0, 1])
    

### Elitismo

In [29]:
def seleccion_elitista(poblacion: list, n: int, fun_aptitud: Callable = aptitud_maximizar) -> list:
    """Selecciona a los n individuos más aptos de la población.
    :param poblacion: individuos de entre los que seleccionar
    :param n: número de individuos a seleccionar
    :param fun_aptitud: función para calcular la aptitud de un individuo
    :return: individuos seleccionados"""

    aptitudes = [fun_aptitud(individuo) for individuo in poblacion]
    return sorted(poblacion, key=aptitudes, reverse=True)[:n-1]


## Prueba de ejecución

In [30]:
# TODO: Implementar el algoritmo genético completo
INDIVIDUOS = 50
PROB_CROSSOVER = 0.25
PROB_MUTACION = 0.01
GENERACIONES = 100

poblacion = crear_poblacion(INDIVIDUOS)

mejores_aptitudes = []
aptitudes_medias = []

for _ in range(GENERACIONES):
    aptitudes_medias.append(sum([aptitud_maximizar(individuo) for individuo in poblacion]) / INDIVIDUOS)
    mejores_aptitudes.append(max([aptitud_maximizar(individuo) for individuo in poblacion]))
    seleccionados = seleccionar_ruleta(poblacion, INDIVIDUOS)
    nuevos = []
    for i in range(0, INDIVIDUOS, 2):
        hijos = crossover(seleccionados[i:i+2], PROB_CROSSOVER)
        mutacion(hijos[0], PROB_MUTACION)
        mutacion(hijos[1], PROB_MUTACION)
        nuevos.extend(hijos)
    poblacion = seleccion_elitista(poblacion, INDIVIDUOS) + nuevos
    
# Grafico con los resultados
import matplotlib.pyplot as plt
plt.plot(mejores_aptitudes, label="Mejor aptitud")
plt.plot(aptitudes_medias, label="Aptitud media")
plt.xlabel("Generaciones")
plt.ylabel("Aptitud")
plt.legend()
plt.show()


ValueError: non-integer stop for randrange()